In [116]:
from aocd import get_data, submit
import numpy as np
import sys
import itertools
from collections import defaultdict, Counter
import re

day = 7
year = 2022

data = get_data(day=day, year=year)

# submit(cube.sum(), part="a", day=day, year=year)

# data

PuzzleLockedError: 2022/07 not available yet

In [112]:
# data = "mjqjpqmgbljsphdztnvjfqwrcgsmlb"
# data = "bvwbjplbgvbhsrlpgdmjqwftvncz"

['D', 'C']